In [3]:
# load python packages
import numpy as np
import pandas as pd
import xarray as xr

# load machine learning packages
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import scale

# # minisom package for SOM  
# import minisom
# from minisom import MiniSom

# Importing data from disk

In [ ]:
path='/storage/group/gsj1/default/COMMON/DATA/'
dust_df = pd.read_csv(path+'saharan_dust_met_vars.csv', index_col='time')

# print out shape of data 
print('Shape of data:', np.shape(dust_df))

# print first 5 rows of data
print(dust_df.head())

# Scaling the variables

As you can see there is a large range of values bewteen variables. To not influence the results as it is the case in many unsupervised machine learning models, it is important to scale them. Many scaling methods exist but I am using the minmax scaler.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler(feature_range = (0,1))
sc.fit(dust_df)
scaled_dust_df=sc.transform(dust_df)

In [ ]:
# Define minisom model
som = MiniSom(x=20, # map size
              y=20, # map size, NxN
              input_len=10, # 10 element input vectors
              sigma=1.0,
              learning_rate=0.5, 
              neighborhood_function='triangle' # a few options for this
             )

# input_len: number of features used for training the model
# sigma: is the radius of the different neighbors in the SOM
# learning rate: determines how much the weights are adjusted during each iterations

# initilize weights using PCA
# You could also do that using random_weights_init, but the advantage is that PCA is likely to converge faster
#som.pca_weights_init(scaled_dust_df)  # prefrerred
som.random_weights_init(scaled_dust_df)

## training the SOM : there are two type of training
# 1. train_random: trains model by pickinhg random data from the data
# 2. train_batch: trains model from samples in the order in which they are fed.

som.train(data = scaled_dust_df, num_iteration = 3000, 
          random_order=True, verbose=True)

# Visualizing the Results

In [4]:
from pylab import bone, pcolor, colorbar, plot, show

In [ ]:
bone()
pcolor(som.distance_map().T)
colorbar()

# markers = ['o', 's']
# colors = ['r', 'g']

# for i, x in enumerate(scaled_dust_df):
#     w = som.winner(x)
#     # w[0], w[1] will place the marker at bottom left corner of the rectangle. 
#     #Let us add 0.5 to both of these to plot the market at the center of the rectange.
#     plot(w[0] + 0.5, 
#          w[1] + 0.5,
#          #Target value 0 will have marker "o" with color "r"
#          #Target value 1 will have marker "s" with color "g"
#          marker='o', 
#          markeredgecolor = 'r',
#          markerfacecolor = 'None', #No color fill inside markers
#          markersize = 10,
#          markeredgewidth = 2)
# show()


# Random Forest Regression

In [ ]:
# Load packages
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#Model evaluation
from sklearn.metrics import mean_squared_error,mean_absolute_error

# Define a function for error calculations

In [ ]:
def regression_stats(model_fit, model_name, features_test, pred, target_test):
  #Calculate and display model error
  score = model_fit.score(features_test,target_test)
  print('\n'+model_name)
  print(f'Score : {score}')
  print(f'MAE: {mean_absolute_error(pred,target_test)}')
  print(f'RMSE : {np.sqrt(mean_squared_error(pred,target_test))}')

In [ ]:
target_vars = dust_df['PM10']   # PM10 concentration is the target variable
features = dust_df.drop(['PM10'], axis=1)  # remaining variables should be features

# split the data into 70% training and reserve 30% for testing
train_features, test_features, train_target, test_target = train_test_split(features, 
                                                                target_vars, test_size = 0.3)

# Now train the model using the training sets
rf = RandomForestRegressor(n_estimators=200,
                    random_state=42, n_jobs=-1)
# Fit Random forest model
rf.fit(train_features, train_target)
predictions = rf.predict(test_features)

regression_stats(rf,'Random Forest',test_features, predictions, test_target)

# Variable importance

In [ ]:
# customize figure 
import matplotlib as mpl
mpl.rcParams['font.size'] = 28
mpl.rcParams['legend.fontsize'] = 'large'
mpl.rcParams['figure.titlesize'] = 'large'
mpl.rcParams['lines.linewidth'] = 2.5
mpl.rcParams['axes.linewidth'] = 2.5
mpl.rcParams["axes.unicode_minus"] = True
mpl.rcParams['figure.dpi'] = 150
mpl.rcParams['savefig.bbox']='tight'
mpl.rcParams['hatch.linewidth'] = 2.5

In [ ]:
col_names = list(features.columns)
cols = np.array(['red', 'blue', 'green', 'cyan', 'pink', 'olive', 'purple', 'magenta',
                'indigo'])

fig, ax = plt.subplots(1,1, figsize=(22,14), sharex=False, sharey=False, 
                               constrained_layout=True) 

sorted_idx = rf.feature_importances_.argsort()
importances = rf.feature_importances_
x_values = col_names

# Make a bar chart
ax.barh(features.columns[sorted_idx], 
           importances[sorted_idx], color=cols)

ax.set_xlabel('Random Forest Feature Importance')     
ax.set_ylabel('Features')

# Hyper-parameter tuning?